In [104]:
# reads and processes a single file from song_data and log_data and loads the data into your tables. 
# This notebook contains detailed instructions on the ETL process for each of the tables.
# Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` 
#file to load the whole datasets.
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json
from datetime import datetime
import numpy as np

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=mydb user=AdminUser password=PassWord123")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
# Get a list of all song JSON files in data/song_data
song_files = get_files('/Users/doopyduper/Documents/DevOps/Course-Exploring_NoSQL_Databases/Code/PostgreSQL/lesson2/project/data/song_data')

In [5]:
# Select the first song in the list
filepath = song_files[0]
filepath

'/Users/doopyduper/Documents/DevOps/Course-Exploring_NoSQL_Databases/Code/PostgreSQL/lesson2/project/data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [6]:
# Read the song file
df = pd.DataFrame(json.load(open(filepath)), index=[0])
# View the data
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,None,None,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [7]:
# Select song_id, title, artist_id, year, and duration
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
# Select the first element in the array
song_data = song_data.values[0]
# Convert to list
song_data = song_data.tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [10]:
# Select artist_id, name, location, latitude, and longitude
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
# Select the first element in the array
artist_data = artist_data.values[0]
# Convert to list
artist_data = artist_data.tolist()
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None]

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [12]:
log_files = get_files('/Users/doopyduper/Documents/DevOps/Course-Exploring_NoSQL_Databases/Code/PostgreSQL/lesson2/project/data/log_data')

In [13]:
filepath = log_files[0]
filepath

'/Users/doopyduper/Documents/DevOps/Course-Exploring_NoSQL_Databases/Code/PostgreSQL/lesson2/project/data/log_data/2018/11/2018-11-11-events.json'

In [100]:
# Read the song file
df = pd.read_json(filepath, lines=True)
# View the data
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
- Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
- Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [101]:
# Filter records by NextSong action
next_song = df['page']=='NextSong'
df = df[next_song]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [109]:
# Convert the ts timestamp column to datetime
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [127]:
#  extract timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
timestamp = t.astype(np.int64)
hour = t.dt.hour
day = t.dt.day
week = t.dt.week
month = t.dt.month
year = t.dt.year
#print(timestamp.dtype, hour.dtype, day.dtype, week.dtype, month.dtype, year.dtype)
time_data = pd.Series([t.astype(np.int64), t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year])
# Specify labels for these columns and set to column_labels
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [144]:
# Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe
time_dict = dict(zip(column_labels, time_data))
time_df = pd.DataFrame(time_dict)
time_df.head()

,timestamp,hour,day,week,month,year
0,1541903636796000000,2,11,45,11,2018
1,1541903770796000000,2,11,45,11,2018
2,1541904034796000000,2,11,45,11,2018
4,1541910973796000000,4,11,45,11,2018
5,1541911006796000000,4,11,45,11,2018


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [153]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [157]:
# user ID, first name, last name, gender and level
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


In [159]:
# insert records for the users in this log file into the users table
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artistname, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artistname, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [161]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

SyntaxError: invalid syntax (<ipython-input-161-2ea5cadf1573>, line 15)

In [162]:
conn.close()conn.close()

SyntaxError: invalid syntax (<ipython-input-162-1bef2c7371fe>, line 1)